## IMPORT

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import fmin_l_bfgs_b 
from cvxopt import matrix, solvers
import pickle as pkl
from scipy import optimize
from scipy.linalg import cho_factor, cho_solve

In [3]:
Xtr = np.array(pd.read_csv('data_image/Xtr.csv',header=None,sep=',',usecols=range(3072))) 
Xte = np.array(pd.read_csv('data_image/Xte.csv',header=None,sep=',',usecols=range(3072))) 
Ytr = np.array(pd.read_csv('data_image/Ytr.csv',sep=',',usecols=[1])).squeeze() 

## VIZUALISATION

Let's visualize data :

In [ ]:
def plot_images_grid(data, nrows, ncols):
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*2, nrows*2))
    random=np.random.choice(data.shape[0],size=nrows*ncols)
    #data=(data-np.min(data))/(np.max(data)-np.min(data))
    for j, ax in enumerate(axes.flat):
        i=random[j]
        if i < data.shape[0]:
            image_data = data[i, :]
            
            # Normaliser les données dans l'intervalle [0, 1]
            # min_val = image_data.min()
            # max_val = image_data.max()
            # print(min_val,max_val)
            # image_data = (image_data - min_val) / (max_val - min_val)
            # print(image_data.max())
            
            red_channel = image_data[:1024].reshape((32, 32))
            red_channel=(red_channel-red_channel.min())/(red_channel.max()-red_channel.min())
            green_channel = image_data[1024:2048].reshape((32, 32))
            green_channel=(green_channel-green_channel.min())/(green_channel.max()-green_channel.min())
            blue_channel = image_data[2048:].reshape((32, 32))
            blue_channel=(blue_channel-blue_channel.min())/(blue_channel.max()-blue_channel.min())
            image = np.stack((red_channel, green_channel, blue_channel), axis=-1)

            ax.imshow(image)
            ax.axis('off')
        else:
            ax.axis('off')

    plt.tight_layout()
    plt.show()
plot_images_grid(Xtr,2,2)

## KERNELS

Let's define some kernels :

In [4]:
class RBF():
    """
    Compute the matrix of the Gaussian Kernel : 
    
    (K)_ij=K(X_i,Y_j)=exp( 1/2*sigma^2 * ||X_i-Y_j||^2 ) the Gaussian Kernel evaluated between the ith data and jth data

    Parameters : 
    X : 2d array size (n,p) 
        the Data matrix with n the number of data and p the size of data
    Y : 2d array size (q,p) 
        the Data matrix with q the number of data and p the size of data
    sigma : float 
             Variance of the GaussianKernel 

    Outputs :
    2d array size (n,q)
    Matrix of the Gaussian Kernel
    """   
    def __init__(self, sigma=1.):
        self.sigma = sigma  ## the variance of the kernel
    def kernel(self,X,Y):
        ## Input vectors X and Y of shape Nxd and Mxd
        diff2 = np.sum(X**2, axis=1)[:, None] + np.sum(Y**2, axis=1)[None, :] - 2 * np.dot(X, Y.T)
        return  np.exp(-diff2/(2*self.sigma**2)) ## Matrix of shape NxM

In [5]:
class polynomial():
    """
    Compute the matrix of the Polynomial Kernel : 
    
    (K)_ij=K(X_i,Y_j)=(<X_i,Y_j>)^d the Polynomial Kernel of degree d evaluated between the ith data and jth data

    Parameters : 
    X : 2d array size (n,p) 
        the Data matrix with n the number of data and p the size of data
    Y : 2d array size (q,p) 
        the Data matrix with q the number of data and p the size of data
    d : Integer
            Degree of the polynomial kernel

    Outputs :
    2d array size (n,q)
    Matrix of the Gaussian Kernel
    """
    def __init__(self,d=2):
        self.d=d

    def kernel(self,X,Y):
        return np.dot(X,Y.T) ** self.d
        # X_intercept = np.concatenate((X,np.ones(X.shape[0]).reshape(-1,1)), axis=1)
        # Y_intercept= np.concatenate((Y,np.ones(Y.shape[0]).reshape(-1,1)), axis=1)
        # return np.sum(X_intercept * Y_intercept[:,None,:], axis=-1) ** self.d

## MODELS

In [ ]:
class Kernel_ridge_reg():
    """
    Class which comput the solution to the Kernel Ridge Regression with regularization parameter lambda (lmbda)

    ----> We have a close form for the solution : 
            alpha=(K+lambda*n*I)^-1y
            pred=K@alpha
    """

    def __init__(self,lmbda):
        self.lmbda=lmbda

    def train(self,K,y):
        mat=K+self.lmbda*K.shape[0]*np.identity(K.shape[0])
        self.alpha=scipy.linalg.solve(mat,y)
    
    def fit(self,K):
        print(self.alpha.shape,K.shape)
        return self.alpha@K

In [ ]:
class Kernel_logistic_reg():
    """
    Class which comput the solution to the Kernel logistic Regression with regularization parameter lambda (lmbda)

    ----> We have to solve the smooth convex opti problem : 
            alpha=argmin 1/n sum_i=1^n logisitic(y_i[Kalpha]_i)+lambda/2 *alpha^T@K@alpha

          We solve this with the L-FBGS form scipy

          and then : 
            pred=K@alpha
    """

    def __init__(self,lmbda,alpha0):
        self.lmbda=lmbda
        self.alpha0=alpha0

    def obj(self,K,y,alpha):
        n=len(K)
        return np.sum(np.log(1+np.exp(-y*(K@alpha))))/n+self.lmbda*alpha.T@K@alpha/2

    def derivative(self,K,y,alpha):
        n=len(K)
        P=-np.diag(1/(1+np.exp(y*(K@alpha))))
        return K@P@y/n+self.lmbda*K@alpha

    def train(self,K,y):
        ob= lambda alpha : self.obj(K=K,y=y,alpha=alpha)
        der=lambda alpha : self.derivative(K=K,y=y,alpha=alpha)
        alpha,_,_=fmin_l_bfgs_b(ob, self.alpha0, der, args=(), pgtol=1e-50, factr =1e-50)
        self.alpha=alpha
    
    def fit(self,K):
        return K@self.alpha

In [ ]:
class SVM():
     """
    Class which train SVM models with Kernels methods takes a dataset with labels in {-1,1} 

    """
     def __init__(self,lmbda):
          self.lmbda = lmbda

     def train(self,K,y):
          """ 
          We want to solve max mu^T@1 - 1/4*lambda mu^t@diag(y)@K@diag(y)@mu for 0<=mu<=1/n
          and then we have :
               alpha=diag(y)@mu/2*lambda

          We use the package cvxopt wich solve : 
               min 1/2x^T@P@x + q^T@x for Gx<=h and Ax=b
          """
          n=len(K)
          
          q = -np.ones(n)
          P = np.diag(y) @ K @np.diag(y) / (2*self.lmbda)
          G = np.concatenate((np.identity(n),-np.identity(n)),axis=0)
          h = np.concatenate((np.ones(n)/n , np.zeros(n)),axis=0)[:,None]

          mu=solvers.qp(P=matrix(P),q=matrix(q),G=matrix(G),h=matrix(h))['x']
          
          self.alpha=np.diag(y)@mu / (2*self.lmbda)

     
     def fit(self,K):
          return np.sign(K@self.alpha)
     
     def pred_prob(self,K):
          return K@self.alpha
     


In [ ]:
class KernelSVC:

    """
    Class which train SVM models with Kernels methods takes a dataset with labels in {-1,1} 

    """
    
    def __init__(self, C, kernel, epsilon = 1e-1):
        self.type = 'non-linear'
        self.C = C                               
        self.kernel = kernel        
        self.alpha = None
        self.support = None
        self.epsilon = epsilon 
        self.norm_f = None
       
    
    def fit(self, X, y):
        N = len(y)
        K=self.kernel(X,X)
        print('kernel computed')
        diag=np.diag(y)

        # Lagrange dual problem
        def loss(alpha):
            return  (1/2)*(diag@alpha).T@K@(diag@alpha)-np.sum(alpha) 

        # Partial derivate of Ld on alpha
        def grad_loss(alpha):
            return diag@K@diag@alpha-np.ones_like(alpha) 


        # Constraints on alpha of the shape :
        # -  d - C*alpha  = 0
        # -  b - A*alpha >= 0

        fun_eq = lambda alpha: alpha.T@y  # '''----------------function defining the equality constraint------------------'''        
        jac_eq = lambda alpha: y   #'''----------------jacobian wrt alpha of the  equality constraint------------------'''
        fun_ineq = lambda alpha: np.concatenate((alpha,self.C-alpha))   # '''---------------function defining the inequality constraint-------------------'''     
        jac_ineq = lambda alpha:  np.concatenate((np.identity(len(alpha)),-np.identity(len(alpha)))) # '''---------------jacobian wrt alpha of the  inequality constraint-------------------'''
        
        constraints = ({'type': 'eq',  'fun': fun_eq, 'jac': jac_eq},
                       {'type': 'ineq', 
                        'fun': fun_ineq , 
                        'jac': jac_ineq})
        print('begin opti :')
        optRes = optimize.minimize(fun=lambda alpha: loss(alpha),
                                   x0=np.ones(N), 
                                   method='SLSQP', 
                                   jac=lambda alpha: grad_loss(alpha), 
                                   constraints=constraints,tol=self.epsilon)
        self.alpha = optRes.x
        print('end opti')
        ## Attributes
        indice_sv=np.where(np.abs(self.alpha)>1e-5)[0]
        self.support=X[indice_sv]
        self.alpha_support=self.alpha[indice_sv]
        self.beta=diag@self.alpha
        self.beta_support=self.beta[indice_sv]
        margin_indices = np.where((self.alpha > 1e-5) & (self.alpha < self.C-1e-5))[0]
        self.margin_points = X[margin_indices]  #'''------------------- A matrix with each row corresponding to a point that falls on the margin ------------------'''
        self.b = np.mean(y[indice_sv]-(K@self.beta)[indice_sv])  #''' -----------------offset of the classifier------------------ '''
        self.norm_f = self.beta[indice_sv].T@(K@self.beta)[indice_sv]# '''------------------------RKHS norm of the function f ------------------------------'''

 
    def separating_function(self,x):
        # Input : matrix x of shape N data points times d dimension
        # Output: vector of size N
        return self.kernel(x,self.support)@self.beta_support
    
    
    def predict(self, X):
        """ Predict y values in {-1, 1} """
        d = self.separating_function(X)
        return 2 * (d+self.b> 0) - 1

In [55]:
class KernelSVC_cvxopt:

    
    def __init__(self, C, kernel, epsilon = 1e-3):
        self.type = 'non-linear'
        self.C = C                               
        self.kernel = kernel        
        self.alpha = None
        self.support = None
        self.epsilon = epsilon 
        self.norm_f = None
       
    
    def fit(self, X, y):
       #### You might define here any variable needed for the rest of the code
        N = len(y)
        K=self.kernel(X,X)
        diag=np.diag(y)

        P = diag@K@diag
        q = -np.ones(N)

        G1 = -np.identity(N)
        h1 = np.zeros(N)

        # Pour alpha <= C
        G2 = np.identity(N)
        h2 = np.ones(N) * self.C

        # Combiner en matrices G et h pour cvxopt
        G = np.vstack([G1, G2])
        h = np.hstack([h1, h2])
        
        
        self.alpha = solvers.qp(P=matrix(P),q=matrix(q),G=matrix(G),h=matrix(h))['x']
        self.alpha = np.array(self.alpha).reshape(-1)
        
        ## Assign the required attributes

        indice_sv=np.where(np.abs(self.alpha)>1e-5)[0]
        #print(indice_sv)
        #print(np.array(self.alpha).reshape(-1))
        
        self.support=X[indice_sv]
        self.alpha_support=self.alpha[indice_sv]

        self.beta=diag@self.alpha
        self.beta_support=self.beta[indice_sv]

        margin_indices = np.where((self.alpha > 1e-5) & (self.alpha < self.C-1e-5))[0]
        self.margin_points = X[margin_indices]  #'''------------------- A matrix with each row corresponding to a point that falls on the margin ------------------'''
        
        self.b = np.mean(y[indice_sv]-(K@self.beta)[indice_sv])  #''' -----------------offset of the classifier------------------ '''

        self.norm_f = self.beta[indice_sv].T@(K@self.beta)[indice_sv]# '''------------------------RKHS norm of the function f ------------------------------'''


    ### Implementation of the separting function $f$ 
    def separating_function(self,x):
        # Input : matrix x of shape N data points times d dimension
        # Output: vector of size N
        return self.kernel(x,self.support)@self.beta_support
    
    
    def predict(self, X):
        """ Predict y values in {-1, 1} """
        d = self.separating_function(X)
        return 2 * (d+self.b> 0) - 1

## UTILS FUNCTION

In [ ]:
def create_reduce_dataset(X,Y,size):
    """ 
    Create a dataset of size = 10*size with 10% of each class
    
    and transform the labels of class k in 1 and the labels of the others classes in -1 
    """

    Y_new=np.array([])
    X_new=np.random.randint(2,size=(1,X.shape[1]))

    for i in range(len(np.unique(Y))):
        ind=np.random.choice(np.array(np.where(Y==i))[0],size=size)
        X_new=np.concatenate((X_new,X[ind]))
        Y_new=np.concatenate((Y_new,i*np.ones(size)))
    X_new=X_new[1:]
    ind=np.arange(Y_new.shape[0])
    np.random.shuffle(ind)
    Y_shuffled=Y_new[ind]
    X_shuffled=X_new[ind]
    return X_shuffled,Y_shuffled


In [ ]:
def create_reduce_dataset_onevsall(X,Y,k,size):
    """ 
    Create a dataset of size = 18*size with 50% of class k and 50% random classes  
    
    and transform the labels of class k in 1 and the labels of the others classes in -1 
    """

    Y_new=np.array([])
    X_new=np.random.randint(2,size=(1,X.shape[1]))

    for i in range(len(np.unique(Y))):
        if i == k :
            ind=np.random.choice(np.array(np.where(Y==i))[0],size=size*9)
        else :
            ind=np.random.choice(np.array(np.where(Y==i))[0],size=size)
        X_new=np.concatenate((X_new,X[ind]))
        if i==k:
            Y_new=np.concatenate((Y_new,np.ones(size*9)))
        else :
            Y_new=np.concatenate((Y_new,-np.ones(size)))
            
    X_new=X_new[1:]
    ind=np.arange(Y_new.shape[0])
    np.random.shuffle(ind)
    Y_shuffled=Y_new[ind]
    X_shuffled=X_new[ind]
    return X_shuffled,Y_shuffled

In [6]:
def accuracy(y_pred,y_test):
    return np.sum(y_pred==y_test)/len(y_pred)

## RUN CODE

Let's try with the class k=4 and a dataset of size train : 600 and test : 300 

The goal is that the classifier classify well the class k from the others class 

In [ ]:
X,Y=create_reduce_dataset_onevsall(Xtr,Ytr,k=4,size=50)
X_train=X[:600]
X_test=X[600:900]
Y_train=Y[:600]
Y_test=Y[600:900]
Big_X=np.concatenate((X_train,X_test),axis=0)

Let's compute the Grams Matrix (let's try with polynomial kernel), it takes a lot of time (1min40 for 900x900 kernel) : 

In [ ]:
K = polynomial().kernel(Big_X,X_train)
K_train = K[:len(X_train),:len(X_train)]
K_test = K[:,len(X_train):]

In [ ]:
K_train.shape,K_test.shape

Let's try differents models : 

In [ ]:
# RIDGE REG not very appropriated for the problem (classification != regression )
classifier=Kernel_ridge_reg(lmbda=0.1)

classifier.train(K_train,Y_train) 

Y_pred = classifier.fit(K_test)

accuracy(np.sign(Y_pred),Y_test)

In [ ]:
# Logistic REG 
classifier=Kernel_logistic_reg(lmbda=0.1,alpha0=np.random.rand(len(X_train)))

classifier.train(K_train,Y_train)

Y_pred = classifier.fit(K_test.T)

accuracy(np.sign(Y_pred),Y_test)

In [ ]:
classifier=SVM(lmbda=0.2)

classifier.train(K_train,Y_train)

Y_pred = classifier.fit(K_test.T)
print(classifier.alpha)

accuracy(Y_pred[:,0],Y_test)

In [ ]:
K=RBF().kernel
classifier=KernelSVC(C=1,kernel=K)
classifier.fit(X_train,Y_train)
Y_pred=classifier.predict(X_test)
accuracy(Y_pred,Y_test)

Let's try to do a loop and attributing an SVM for all classes : 
ONE VS REST strategy 
ONE VS ONE strategy 

### NYSTROM APPROXIMATION

https://github.com/fredhallgren/nystrompca/blob/develop/nystrompca/algorithms/nystrom_KPCA.py

In [ ]:
class NystromKPCA():
    """
    Compute an approximation of kernel with the PCA techniques, 
    taking m data points, projecting on p directions (p<=m)

    """
    def __init__(self,kernel,p,m) :
       self.kernel=kernel
       self.m=m
       self.p=p
       self.alphas=None
       self.big_approximated_kernel=None
       self.big_approximated_repr=None
       self.X_subset=None

    def fit_PCA(self, X):
        # assigns the vectors
        self.X_subset=self.choose_subset(X)

        K=self.kernel(self.X_subset,self.X_subset)

        #We have to center the kernel
        #center = (I-U)K(I-U)
        
        U=np.ones(K.shape)
        I=np.eye(K.shape[0])
        K=(I-U)@K@(I-U)
        
        valp,vectp=np.linalg.eigh(K)

        #we take the last r eingenvalues
        lmbda=valp[-self.p:]
        inverse=1/np.sqrt(lmbda)

        self.alphas=(inverse*vectp[:,-self.p:]).T
    

    def approximated_repr(self,x):
        return self.alphas@self.kernel(self.X_subset,x)

    def appro_kernel(self,X,Y):
        repr_X=self.alphas@self.kernel(self.X_subset,X)
        repr_Y=self.alphas@self.kernel(self.X_subset,Y)
        return repr_X.T@repr_Y

    def choose_subset(self,X):
        self.n=X.shape[0]
        set=np.random.choice(self.n,self.m)
        self.X_subset=X[set]
        return set

In [57]:
def create_dataset_onevsall(Y,k):
    """  
    Transform the labels of class k in 1 and the labels of the others classes in -1 
    """
    Y_onevall=-np.ones_like(Y)
    Y_onevall[Y==k]=1
    return Y_onevall

In [ ]:
Y_onevall=create_dataset_onevsall(Ytr,1)
Xtr[:100,:].shape,Y_onevall[:100].shape

In [ ]:
kernel=RBF().kernel
classifier=KernelSVC(C=1,kernel=kernel)
classifier.fit(Xtr[:2000,:],Y_onevall[:2000])

In [ ]:
np.abs(classifier.separating_function(Xtr[:2000,:]))

In [ ]:
l=classifier.predict(Xtr[2000:3000,:])==Y_onevall[2000:3000]
np.sum(l)/len(l)

In [ ]:
classifier.alpha

## ONE vs REST

In [ ]:
from tqdm import tqdm
SVMModels = []

X_reduce,Y_reduce = create_reduce_dataset(Xtr,Ytr,150)

# Étape 4-8: Générer N modèles de classe binaire
for j in tqdm(range(10)):

    Y = create_dataset_onevsall(Y_reduce,k=j)
    
    kernel=RBF().kernel
    classifier=KernelSVC(C=1,kernel=kernel)
    
    classifier.fit(X_reduce,Y_reduce)
    
    # Stocker le modèle SVM entraîné
    SVMModels.append(classifier)


In [ ]:
x_test,y_test = create_reduce_dataset(Xtr,Ytr,20)
x_test=X_reduce

# Initialisation d'un dictionnaire pour stocker les scores pour chaque classe
scores = np.zeros((10,x_test.shape[0]))

# Étape 9-12: Calculer les scores pour chaque classe
for j, svm_model in enumerate(SVMModels):
    # Prédire le score pour la classe actuelle
    score = svm_model.separating_function(x_test)
    
    # Stocker le score pour la classe actuelle
    scores[j] = score

# Étape 13: Attribuer à chaque observation la classe avec le score le plus élevé
final_classes = np.argmax(scores,axis=0)+1
accuracy(final_classes,Y_reduce)

In [ ]:
SVMModels[0].alpha[116]
SVMModels[0].beta[421]

## APPRO DE LA DIM PAR PCA 

In [ ]:
class KernelPCA:
    
    def __init__(self,kernel, r=2):                             
        self.kernel = kernel          # <---
        self.alpha = None # Matrix of shape N times d representing the d eingenvectors alpha corresp
        self.lmbda = None # Vector of size d representing the top d eingenvalues
        self.support = None # Data points where the features are evaluated
        self.r =r ## Number of principal components

    def compute_PCA(self, X):
        # assigns the vectors
        
        self.support = X
        K=self.kernel(X,X)

        #We have to center the kernel
        #center = (I-U)K(I-U)
        
        U=np.ones(K.shape)
        I=np.eye(K.shape[0])
        K=(I-U)@K@(I-U)
        
        valp,vectp=np.linalg.eigh(K)

        #we take the last r eingenvalues
        self.lmbda=valp[-self.r:]
        inverse=1/np.sqrt(self.lmbda)

        self.alpha = inverse*vectp[:,-self.r:]

        
        #constraints = ({})
        # Maximize by minimizing the opposite
        
    def transform(self,x):
        # Input : matrix x of shape N data points times d dimension
        # Output: vector of size N
        K=self.kernel(x,self.support)
        return K@self.alpha

In [ ]:

kernel=RBF().kernel
PCA=KernelPCA(kernel=kernel,r=1000)
PCA.compute_PCA(Xtr) # We choose the most dim of X
X=PCA.transform(Xtr)

print(X.shape)


#classifier.fit(Xtr[:1000,0:10],Y[:1000])

In [ ]:
Y = create_dataset_onevsall(Ytr,k=3)

kernel=RBF().kernel
classifier=KernelSVC(C=1,kernel=kernel)

classifier.fit(Xtr,Y)

In [ ]:
from tqdm import tqdm
SVMModels = []

# Étape 4-8: Générer N modèles de classe binaire
for j in tqdm(range(10)):

    Y = create_dataset_onevsall(Ytr,k=j)
    
    kernel=RBF().kernel
    classifier=KernelSVC(C=1,kernel=kernel)
    
    classifier.fit(X,Y)
    
    # Stocker le modèle SVM entraîné
    SVMModels.append(classifier)

## Kernel Logistic reg

In [ ]:
class Kernel_logistic_reg():
    """
    Class which comput the solution to the Kernel logistic Regression with regularization parameter lambda (lmbda)

    ----> We have to solve the smooth convex opti problem : 
            alpha=argmin 1/n sum_i=1^n logisitic(y_i[Kalpha]_i)+lambda/2 *alpha^T@K@alpha

          We solve this with the L-FBGS form scipy

          and then : 
            pred=K@alpha
    """

    def __init__(self,lmbda,alpha0):
        self.lmbda=lmbda
        self.alpha0=alpha0

    def obj(self,K,y,alpha):
        n=len(K)
        return np.sum(np.log(1+np.exp(-y*(K@alpha))))/n+self.lmbda*alpha.T@K@alpha/2
    
    def sigmoid(self,u):
        return 1 / (1 + np.exp(-u))

    def logistic(self,u):
        return np.log(1 + np.exp(-u))


    def logistic_prime(self,u):
        return -self.sigmoid(-u)


    def logistic_prime2(self,u):
        return self.sigmoid(u) * self.sigmoid(-u)


    def derivative(self,K,y,alpha):
        n=len(K)
        P=-np.diag(1/(1+np.exp(y*(K@alpha))))
        return K@P@y/n+self.lmbda*K@alpha
    
    def second_derivative(self,K,y,alpha):
       n=len(K)
       W=np.diag(self.logistic_prime2(y*(K@alpha)))
       return (1/n)* K@W@K + self.lmbda*K

    def train(self,K,y):
        ob= lambda alpha : self.obj(K=K,y=y,alpha=alpha)
        der=lambda alpha : self.derivative(K=K,y=y,alpha=alpha)
        secder=lambda alpha : self.second_derivative(K=K,y=y,alpha=alpha)
        
        optRes = optimize.minimize(fun=ob,
                                   x0=np.ones(len(K)), 
                                   method='SLSQP', 
                                   jac=der,hess=secder
                                   ,tol=1e-5)
        self.alpha = optRes.x
    
    def fit(self,K):
        return self.sigmoid(K@self.alpha)

In [ ]:
class KernelLogisticRegression:

    def __init__(self, kernel, reg_param=0, epsilon=1e-8):
        self.alpha = None
        self.reg_param = reg_param
        self.beta = None
        self.kernel = kernel
        self.eps = epsilon
        self.support=None

    def fit(self, X, y):
        N = X.shape[0]
        self.support=X
        #features_X = self.kernel.fit_subtree(X)
        k = self.kernel(X,X)
        alpha = np.zeros(N)
        alpha_old = alpha + np.inf
        sig = np.vectorize(sigmoid)
        logpp = np.vectorize(logistic_prime2)
        i=0
        while (np.abs(alpha - alpha_old) > self.eps).any():
            # Update coefs
            m = k @ alpha
            W = np.diag(logpp(y * m))
            z = m + y / sig(y * m)

            # Solve Weighted KRR

            sqrt_W = np.sqrt(W)

            alpha_old = alpha
            alpha = sqrt_W @ np.linalg.inv(
                sqrt_W @ k @ sqrt_W + N * self.reg_param * np.eye(N)
            ) @ sqrt_W @ z
            print(f'{i}ème iteration, epsilon :{np.max(np.abs(alpha - alpha_old))}')
            i+=1
            if i ==10 :
                break

        self.alpha = alpha

        return sigmoid(np.einsum('i, ij->j', self.alpha, k))

    def predict(self, X):
        # features_pred = self.kernel.predict(X)
        # print(features_pred.shape, self.features.shape)
        K_Xx = self.kernel(X, self.support)
        predictions = sigmoid(np.einsum('i, ij->j', self.alpha, K_Xx.T))
        return predictions  # *-1 because inverted prediction on 1 et -1


def logistic(u):
    return np.log(1 + np.exp(-u))


def logistic_prime(u):
    return -sigmoid(-u)


def logistic_prime2(u):
    return sigmoid(u) * sigmoid(-u)


def sigmoid(u):
    return 1 / (1 + np.exp(-u))

In [ ]:
Y = create_dataset_onevsall(Ytr,k=1)
    
kernel=RBF().kernel
classifier=KernelLogisticRegression(kernel=kernel)
classifier.fit(Xtr,Y)

In [ ]:
Y = create_dataset_onevsall(Ytr,k=1)
    
kernel=RBF().kernel
classifier=KernelLogisticRegression(kernel=kernel)
classifier.fit(Xtr,Y)

In [ ]:
pred=np.zeros(5000)
pred[classifier.predict(Xtr)>0.5]=1
pred[classifier.predict(Xtr)<=0.5]=-1
accuracy(pred,Ytr)

In [ ]:
Y = create_dataset_onevsall(Ytr,k=1)
    
kernel=RBF().kernel
classifier=Kernel_logistic_reg(0.01,np.zeros(len(Xtr)))
K=kernel(Xtr,Xtr)
classifier.train(K,Ytr)

## ONE VS ALL

In [58]:
from tqdm import tqdm
LogModels = []

# Étape 4-8: Générer N modèles de classe binaire
for j in tqdm(range(10)):

    Y = create_dataset_onevsall(Ytr,k=j)
    
    kernel=RBF().kernel
    classifier=KernelSVC_cvxopt(C=1,kernel=kernel)
    
    classifier.fit(Xtr,Y)
    
    # Stocker le modèle SVM entraîné
    LogModels.append(classifier)


  0%|          | 0/10 [00:00<?, ?it/s]

     pcost       dcost       gap    pres   dres
 0: -1.2284e+03 -1.5681e+04  1e+05  4e+00  2e-15
 1: -1.0770e+03 -1.1653e+04  2e+04  5e-01  2e-15
 2: -9.5063e+02 -3.1508e+03  2e+03  2e-16  4e-15
 3: -1.0454e+03 -1.3696e+03  3e+02  2e-16  2e-15
 4: -1.0902e+03 -1.1439e+03  5e+01  2e-16  1e-15
 5: -1.1009e+03 -1.1057e+03  5e+00  2e-16  1e-15
 6: -1.1022e+03 -1.1025e+03  3e-01  2e-16  1e-15
 7: -1.1023e+03 -1.1023e+03  9e-03  2e-16  1e-15
 8: -1.1023e+03 -1.1023e+03  3e-04  2e-16  1e-15
Optimal solution found.


 10%|█         | 1/10 [00:38<05:49, 38.85s/it]

     pcost       dcost       gap    pres   dres
 0: -9.0844e+02 -1.3681e+04  8e+04  3e+00  2e-15
 1: -7.8090e+02 -9.6627e+03  2e+04  5e-01  2e-15
 2: -5.8834e+02 -2.3596e+03  2e+03  2e-16  1e-14
 3: -7.0672e+02 -9.9674e+02  3e+02  2e-16  3e-15
 4: -7.4963e+02 -8.0189e+02  5e+01  2e-16  1e-15
 5: -7.6220e+02 -7.6828e+02  6e+00  2e-16  1e-15
 6: -7.6419e+02 -7.6457e+02  4e-01  2e-16  1e-15
 7: -7.6435e+02 -7.6436e+02  1e-02  2e-16  1e-15
 8: -7.6435e+02 -7.6435e+02  4e-04  2e-16  1e-15
Optimal solution found.


 20%|██        | 2/10 [01:17<05:10, 38.80s/it]

...

 80%|████████  | 8/10 [05:16<01:20, 40.17s/it]

     pcost       dcost       gap    pres   dres
 0: -1.2628e+03 -1.4031e+04  8e+04  3e+00  3e-15
 1: -1.1118e+03 -1.0087e+04  2e+04  4e-01  2e-15
 2: -1.0070e+03 -2.6240e+03  2e+03  2e-16  5e-15
 3: -1.0897e+03 -1.3244e+03  2e+02  2e-16  2e-15
 4: -1.1276e+03 -1.1629e+03  4e+01  2e-16  2e-15
 5: -1.1355e+03 -1.1395e+03  4e+00  1e-16  2e-15
 6: -1.1367e+03 -1.1369e+03  2e-01  2e-16  2e-15
 7: -1.1367e+03 -1.1367e+03  6e-03  1e-16  2e-15
 8: -1.1367e+03 -1.1367e+03  2e-04  2e-16  2e-15
Optimal solution found.


 90%|█████████ | 9/10 [05:57<00:40, 40.24s/it]

     pcost       dcost       gap    pres   dres
 0: -9.4416e+02 -1.3762e+04  8e+04  3e+00  2e-15
 1: -8.1362e+02 -9.7466e+03  2e+04  5e-01  2e-15
 2: -6.2272e+02 -2.4170e+03  2e+03  2e-16  9e-15
 3: -7.4023e+02 -9.9916e+02  3e+02  2e-16  3e-15
 4: -7.8351e+02 -8.2424e+02  4e+01  2e-16  2e-15
 5: -7.9431e+02 -7.9915e+02  5e+00  2e-16  1e-15
 6: -7.9597e+02 -7.9621e+02  2e-01  2e-16  1e-15
 7: -7.9608e+02 -7.9609e+02  8e-03  2e-16  1e-15
 8: -7.9608e+02 -7.9608e+02  3e-04  2e-16  1e-15
Optimal solution found.


100%|██████████| 10/10 [06:37<00:00, 39.76s/it]


In [96]:
x_test = Xte

# Initialisation d'un dictionnaire pour stocker les scores pour chaque classe
scores = np.zeros((10,x_test.shape[0]))

# Étape 9-12: Calculer les scores pour chaque classe
for j, log_model in enumerate(LogModels):
    # Prédire le score pour la classe actuelle
    score = log_model.separating_function(x_test)
    
    # Stocker le score pour la classe actuelle
    scores[j] = score

# Étape 13: Attribuer à chaque observation la classe avec le score le plus élevé
final_classes = np.argmax(scores,axis=0)
final_classes

array([1, 8, 2, ..., 4, 2, 4])

In [101]:
np.sum(final_classes==5)

108

In [102]:
import pandas as pd

# Créer un DataFrame à partir des prédictions
# Remplacez 'predictions' par vos données de prédiction
df = pd.DataFrame({
    'Id': range(1, 2001),  # Créer une colonne d'ID de 1 à 2000
    'Prediction': final_classes  # Utilisez vos propres résultats de prédiction ici
})

In [103]:
df

,Id,Prediction
0,1,1
1,2,8
2,3,2
3,4,8
4,5,7
...,...,...
1995,1996,8
1996,1997,8
1997,1998,4
1998,1999,2


In [ ]:
df.to_csv('submission3.csv', index=False)

## One vs one 

## KRR 


In [1]:
class KernelRR:
    
    def __init__(self,kernel,lmbda):
        self.lmbda = lmbda                    
        self.kernel = kernel    
        self.alpha = None 
        self.b = None
        self.support = None
        self.type='ridge'
        
    def fit(self, X, y):
        N=len(y)
        self.support = X
        ones=np.ones((N,1))
        K=self.kernel(X,X)
        K_prime=np.block([[K, ones], [ones.T, np.ones((1, 1))]])
        y_prime=np.append(y,[0])

        mat=K_prime+(N/2)*self.lmbda*np.identity(N+1)

        alpha_prime=np.linalg.solve(mat,y_prime)  
        self.alpha = alpha_prime[:-1]  
        self.b=alpha_prime[-1]     
        
    ### Implementation of the separting function $f$ 
    def regression_function(self,x):
        # Input : matrix x of shape N data points times d dimension
        # Output: vector of size N
        K=self.kernel(x,self.support)
        return K@self.alpha


    
    def predict(self, X):
        """ Predict y values in {-1, 1} """
        return self.regression_function(X)+self.b

In [7]:
Xtr.shape

(5000, 3072)

In [53]:
sigma =0.0001
lmbda= 0.01
kernel=polynomial(d=5).kernel
ridge=KernelRR(kernel,lmbda=lmbda)
ridge.fit(Xtr[:4500],Ytr[:4500])

In [54]:
ridge.predict(Xtr)[4490:4510]

array([ 4.82071437,  1.99011693,  8.95661452,  4.93874889,  1.83716675,
        3.59391532, -0.63423097,  4.91184532,  4.27726282,  8.81121122,
       -3.64590529, -3.64537123, -3.65213901, -3.63929281, -3.62814046,
       -3.67154309, -3.60491434, -3.64469469, -3.64716204, -3.63841473])

In [47]:
Ytr[4590:]

array([9, 0, 4, 3, 9, 0, 2, 4, 7, 9, 8, 8, 6, 7, 9, 0, 7, 4, 1, 8, 2, 6,
       5, 7, 5, 8, 5, 2, 8, 0, 7, 1, 8, 7, 2, 7, 9, 8, 9, 5, 9, 7, 5, 7,
       4, 6, 5, 4, 7, 8, 7, 2, 4, 4, 7, 1, 6, 0, 4, 0, 0, 9, 3, 1, 9, 8,
       2, 9, 8, 2, 1, 5, 3, 9, 6, 4, 1, 1, 3, 8, 7, 2, 4, 2, 1, 0, 0, 1,
       2, 5, 3, 6, 3, 1, 5, 2, 4, 6, 8, 2, 6, 9, 7, 4, 5, 9, 6, 4, 2, 9,
       0, 5, 9, 1, 6, 3, 2, 2, 2, 1, 4, 7, 4, 2, 0, 0, 9, 1, 8, 5, 9, 9,
       0, 2, 6, 7, 5, 0, 7, 0, 2, 8, 9, 2, 9, 5, 6, 3, 3, 8, 4, 6, 8, 1,
       5, 3, 1, 8, 5, 6, 0, 9, 8, 7, 7, 3, 5, 1, 2, 7, 6, 8, 8, 0, 8, 6,
       5, 6, 6, 1, 9, 7, 4, 9, 3, 2, 6, 3, 4, 6, 7, 9, 8, 6, 0, 3, 4, 3,
       4, 1, 5, 7, 0, 3, 2, 0, 7, 3, 0, 5, 6, 8, 4, 8, 9, 6, 7, 0, 5, 0,
       5, 8, 0, 6, 8, 7, 3, 7, 7, 0, 4, 4, 4, 3, 3, 3, 8, 0, 4, 2, 3, 3,
       7, 4, 1, 9, 0, 8, 8, 2, 7, 7, 6, 6, 0, 5, 3, 9, 9, 5, 8, 4, 9, 0,
       1, 8, 3, 7, 2, 8, 3, 7, 2, 5, 4, 9, 3, 2, 1, 5, 2, 0, 4, 8, 4, 7,
       4, 2, 9, 2, 3, 7, 9, 5, 5, 4, 6, 6, 6, 7, 9,